Implemented by Federico Zocco, version 16 March 2022

This Colab notebook generates the TFrecords from PASCAL VOC 2012 annotations, which is the format of Trash-ICRA19 and WPBB datasets.

To convert .xml annotations (in PASCAL VOC format) into .tfrecord I used generate_tfrecord.py available here: https://github.com/sglvladi/TensorFlowObjectDetectionTutorial/blob/master/docs/source/scripts/generate_tfrecord.py

To execute generate_tfrecord.py, the Object Detection API is required. I installed it following
https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html#tensorflow-object-detection-api-installation

The .pbtxt file for the label map is taken from https://github.com/tensorflow/models/blob/master/research/object_detection/data/pascal_label_map.pbtxt and modified for the Trash-ICRA19 and WPBB datasets as appropriate.

REFERENCES
[1] https://github.com/sglvladi/TensorFlowObjectDetectionTutorial/blob/master/docs/source/scripts/generate_tfrecord.py
[2] F. Zocco et al., Towards More Efficient EfficientDets and Low-Light Real-Time Marine Debris Detection, https://arxiv.org/pdf/2203.07155.pdf
[3] https://github.com/google/automl/blob/master/efficientdet/tutorial.ipynb
[4]https://openaccess.thecvf.com/content_CVPR_2020/papers/Tan_EfficientDet_Scalable_and_Efficient_Object_Detection_CVPR_2020_paper.pdf

NOTE: Set the runtime with a GPU (as it seems that generate_tfrecord.py uses some CUDA library functions; if you prefer to use a CPU, make sure that it is allowed by generate_tfrecord.py)  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

os.chdir('drive/MyDrive/TensorFlow')
os.getcwd()


'/content/drive/MyDrive/TensorFlow'

Clone the Object Detection API in Google Drive (mounted previously)

In [3]:
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


Move to "research" folder

In [4]:
os.chdir('models/research')

In [5]:
os.getcwd()

'/content/drive/MyDrive/TensorFlow/models/research'

In [11]:
!protoc object_detection/protos/*.proto --python_out=. #command added to fix the issue discussed here: https://stackoverflow.com/questions/44972277/object-detection-api-error-importerror-cannot-import-name-anchor-generator-pb

In [12]:
!cp object_detection/packages/tf2/setup.py .

In [13]:
!python -m pip install --use-feature=2020-resolver .

Processing /content/drive/MyDrive/TensorFlow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=21908001 sha256=57b793b7b86867182dac79a0719bd2da8007d9da752f4230abc47b6864e6f150
  Stored in directory: /tmp/pip-ephem-wheel-cache-3_5ef3n9/wheels/8b/c1/f5/d0b2e6f976cfbb4a6fe40c59ae0a33a68b4fa09cfb88bbae2a
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [14]:
!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-02-15 10:36:46.583794: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0215 10:36:46.917287 139938853771136 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.4s
I0215 10:36:47.162497 139938853771136 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.4s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.52s
I0215 10:36:47.685069 139938853771136 test_u

Now that the Object Detection API is installed, I can use generate_tfrecord.py. Test the execution first as:

In [15]:
!python generate_tfrecord.py --help

usage: generate_tfrecord.py [-h] [-x XML_DIR] [-l LABELS_PATH]
                            [-o OUTPUT_PATH] [-i IMAGE_DIR] [-c CSV_PATH]

Sample TensorFlow XML-to-TFRecord converter

optional arguments:
  -h, --help            show this help message and exit
  -x XML_DIR, --xml_dir XML_DIR
                        Path to the folder where the input .xml files are
                        stored.
  -l LABELS_PATH, --labels_path LABELS_PATH
                        Path to the labels (.pbtxt) file.
  -o OUTPUT_PATH, --output_path OUTPUT_PATH
                        Path of output TFRecord (.record) file.
  -i IMAGE_DIR, --image_dir IMAGE_DIR
                        Path to the folder where the input image files are
                        stored. Defaults to the same directory as XML_DIR.
  -c CSV_PATH, --csv_path CSV_PATH
                        Path of output .csv file. If none provided, then no
                        file will be written.


Now generate a TFrecord for each shard

In [23]:
shard_number = "3"

# Generate training TFrecords, one per shard
!python generate_tfrecord.py --xml_dir={"train/train_shard" + shard_number} --labels_path="labels_folder/Trash_label_map.pbtxt" --output_path={"Trash_TFrecords/train/TrashICRA19-train-" + shard_number + "-of-" + str(3) + ".tfrecord"} --image_dir={"train/train_shard" + shard_number}

Successfully created the TFRecord file: Trash_TFrecords/train/TrashICRA19-train-3-of-3.tfrecord
